In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")
inspector = inspect(engine)
print(inspector.get_table_names())

columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

['measurement', 'station']
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [6]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 

results = []

results = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date.like('%-06-%')).all()


In [7]:
#  3. Convert the June temperatures to a list.
# completed above as a list of lists
type(results)

list

In [8]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_df = pd.DataFrame(results, columns=['date','temperature'])
june_df.set_index(june_df['date'], inplace=True)
june_df

,date,temperature
date,,
2010-06-01,2010-06-01,78.0
2010-06-02,2010-06-02,76.0
2010-06-03,2010-06-03,78.0
2010-06-04,2010-06-04,76.0
2010-06-05,2010-06-05,77.0
...,...,...
2017-06-26,2017-06-26,79.0
2017-06-27,2017-06-27,74.0
2017-06-28,2017-06-28,74.0


In [9]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_summary_df = june_df.describe()
june_summary_df

,temperature
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [10]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.

results = []

results = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date.like('%-12-%')).all()

In [11]:
# 7. Convert the December temperatures to a list.
# completed above as a list of lists
type(results)

list

In [12]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
december_df = pd.DataFrame(results, columns=['date','temperature'])
december_df.set_index(december_df['date'], inplace=True)

In [13]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
december_summary_df = december_df.describe()
december_summary_df

,temperature
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [14]:
# Additional code to create a 'summary' df
june_summary_df = june_summary_df.reset_index()
december_summary_df = december_summary_df.reset_index()

In [15]:
# Additional code to create a 'summary' df
summary_df = june_summary_df.merge(december_summary_df,how='inner',on='index').rename(columns={'index':'measure','temperature_x': 'Jun', 'temperature_y': 'Dec'}).set_index('measure')
summary_df = summary_df[['Jun','Dec']]
summary_df['Difference'] = summary_df['Dec'] - summary_df['Jun']
summary_df['Diff_Percent'] = summary_df['Difference'] / summary_df['Jun']

In [16]:
# Showing 'summary' df
summary_df

,Jun,Dec,Difference,Diff_Percent
measure,,,,
count,1700.000000,1517.000000,-183.000000,-0.107647
mean,74.944118,71.041529,-3.902588,-0.052073
std,3.257417,3.745920,0.488503,0.149966
min,64.000000,56.000000,-8.000000,-0.125000
25%,73.000000,69.000000,-4.000000,-0.054795
50%,75.000000,71.000000,-4.000000,-0.053333
75%,77.000000,74.000000,-3.000000,-0.038961
max,85.000000,83.000000,-2.000000,-0.023529
